In [1]:
import os
import zipfile
import requests
from typing import Tuple
from google.cloud import storage
from datetime import timedelta
from typing import List, Tuple
from prefect import flow, task
from prefect.tasks import task_input_hash, exponential_backoff

In [10]:
def download_data(url: str, output_dir: str) -> str:

    output_dir = os.path.abspath(output_dir)

    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, os.path.basename(url))

    response = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(response.content)

    return file_path

def extract_data(file_path: str, output_dir: str) -> Tuple[str, str]:

    output_dir = os.path.abspath(output_dir)

    os.makedirs(output_dir, exist_ok=True)
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

    extracted_files = []
    for root, _, files in os.walk(output_dir):
        for file in files:
            extracted_files.append(os.path.join(root, file))

    return tuple(extracted_files)

In [11]:
extract_data(download_data('https://github.com/DataTalksClub/nyc-tlc-data/archive/refs/tags/yellow.zip', 'data'), 'data')

('data/yellow.zip', 'data/nyc-tlc-data-yellow/README.md')